In [1]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from model_performances import *
from copy import deepcopy

In [6]:
model_name = 'gemini'
eval_type = 'zero_shot_data'

results = load_data(eval_type, model_name)
tf_results, fa_results = split_by_answer_type(results)
print(len(tf_results), len(fa_results))

tf_by_length = f1_score_by_length(tf_results, score_type=F1_SCORE_TYPE)
print(tf_by_length)

fa_by_length = f1_rouge_by_length(fa_results)
print(fa_by_length)

# output = {'tf': tf_by_length, 'fa': fa_by_length}
# save_dir = f'{STATISTICS_PATH}/{model_name}/{eval_type}'
# with open('data.json', 'w') as f:
#     json.dump(output, f)

1365 1300
{1: 0.575091575091575, 5: 0.5934065934065934, 10: 0.5311355311355311, 15: 0.5421245421245421, 19: 0.5421245421245421, 'all': 0.5567765567765568}


In [9]:
all_stats = []
for model_name in PROMPT_MODEL_NAMES:
    for eval_type in PROMPT_EVAL_TYPES:
        stats_by_domain = {}
        all_domains_qa = []
        for domain in DOMAIN_NAMES:
            meta_info = {MODEL_NAME_KEY: model_name, EVAL_TYPE_KEY: eval_type, DOMAIN_KEY: domain}
            try:
                results_path = f'{RESULTS_PATH}/{model_name}/{eval_type}/{domain}.jsonl'
                qa_objects = open_jsonl(results_path)
                all_domains_qa.extend(qa_objects)
                all_stats.append(stats_qa_objects_ls(qa_objects) | deepcopy(meta_info))
            except Exception as e:
                print(e)
                all_stats.append({TRUE_FALSE_ANSWER: {}, FREE_ANSWER: {}} | deepcopy(meta_info))
        meta_info[DOMAIN_KEY] = ALL_DOMAINS_KEY
        all_stats.append(stats_qa_objects_ls(all_domains_qa) | deepcopy(meta_info))

[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4/visitall.jsonl'


/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an

[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gemini/few_shot_4_cot/visitall.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/blocksworld.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/depots.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/driverlog.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/goldminer.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/results/gpt4/few_shot_4/grippers.jsonl'
[Errno 2] No such file or directory: '/Users/paveldolin/dev/research/current/reasoning_abou

/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/anaconda3/envs/llm_asp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/an

In [10]:
all_stats

[{'true_false_answer': {1: 0.6666666666666666,
   5: 0.6666666666666666,
   10: 0.7619047619047619,
   15: 0.7619047619047619,
   19: 0.6190476190476191,
   'all': 0.6952380952380952},
  'true_false_answer_in_response': {1: 0.6666666666666666,
   5: 0.6666666666666666,
   10: 0.7619047619047619,
   15: 0.7619047619047619,
   19: 0.6190476190476191,
   'all': 0.6952380952380952},
  'free_answer': {1: 0.5139084289745441,
   5: 0.37424421108139827,
   10: 0.46906054148415566,
   15: 0.4702341395250804,
   19: 0.4648291631269748,
   'all': 0.4584552968384306},
  'model_name': 'gemini',
  'eval_type': 'few_shot_4',
  'domain': 'blocksworld'},
 {'true_false_answer': {1: 0.7,
   5: 0.875,
   10: 0.5,
   15: 0.75,
   19: 0.375,
   'all': 0.6428571428571429},
  'true_false_answer_in_response': {1: 0.7,
   5: 0.875,
   10: 0.5,
   15: 0.75,
   19: 0.375,
   'all': 0.6428571428571429},
  'free_answer': {1: 0.7160878078449799,
   5: 0.7610128939161198,
   10: 0.6441793179850389,
   15: 0.717983506

In [11]:
if not os.path.exists(STATISTICS_PATH):
    os.makedirs(STATISTICS_PATH)
save_path = f'{STATISTICS_PATH}/f1_rouge_everything.json'
with open(save_path, 'w') as f:
    json.dump(all_stats, f)